# Импорт необходимых библиотек, определение основной метрки, определние констант

In [ ]:

import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler, MaxAbsScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.decomposition import PCA

from category_encoders import MEstimateEncoder

import matplotlib.pyplot as plt
import seaborn as sns

# from pymorphy3 import MorphAnalyzer
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

import re
import os

from PIL import Image
import textwrap

import torch
import torch.nn as nn
from torchvision import models, transforms as T
from torchvision.models import ResNet50_Weights
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
import scipy.sparse as sp

from sentence_transformers import SentenceTransformer
from catboost import CatBoostRegressor

In [ ]:
def log_mae(y_true, y_pred):
    return np.mean(
        np.abs(np.log(1 + y_true) - np.log(1 + y_pred))
    )

def macro_log_mae(y_true, y_pred):
    return np.mean([
        log_mae(y_true[:, i], y_pred[:, i])
        for i in range(y_true.shape[1])
    ])

In [ ]:
pd.set_option('display.max_colwidth', None)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
RANDOM_STATE = 52

# EDA

In [ ]:
df = pd.read_parquet('data/train.parquet', engine='pyarrow')
test_df = pd.read_parquet('data/test.parquet', engine='pyarrow')

In [ ]:
df.head()

Посмотрим на колонки и размер датасета

In [ ]:
df.info()

In [ ]:
test_df.info()

In [ ]:
target_cols = ['real_weight', 'real_height', 'real_length', 'real_width']

Видим что в состояние товара имеет пропуски, посмотрим на эту колонку подробней

In [ ]:
df['item_condition'].value_counts(normalize=True)

In [ ]:
df['item_condition'].isna().sum() / df.shape[0] * 100

5% данных имеют этот пропуск, при препроцессинге заполним как новую категорию *Неизвестно*

Также заметим, что есть повторяющиеся по смыслу категории, также в препроцессинге объеденим их

Поищем дубликаты

In [ ]:
df.duplicated().sum()

Построим распределение таргета

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

sns.histplot(df["real_length"], kde=True, stat="density", ax=axes[0, 0])
axes[0, 0].set_title("length")

sns.histplot(df["real_weight"], kde=True, stat="density", ax=axes[0, 1])
axes[0, 1].set_title("weight")

sns.histplot(df["real_height"], kde=True, stat="density", ax=axes[1, 0])
axes[1, 0].set_title("height")

sns.histplot(df["real_width"], kde=True, stat="density", ax=axes[1, 1])
axes[1, 1].set_title("width")

plt.tight_layout()
plt.show()

Видим очень длинный правый хвост, построим распределение логарфмов этой величины

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

sns.histplot(np.log(df["real_length"]), kde=True, stat="density", ax=axes[0, 0])
axes[0, 0].set_title("length")

sns.histplot(np.log(df["real_weight"]), kde=True, stat="density", ax=axes[0, 1])
axes[0, 1].set_title("weight")

sns.histplot(np.log(df["real_height"]), kde=True, stat="density", ax=axes[1, 0])
axes[1, 0].set_title("height")

sns.histplot(np.log(df["real_width"]), kde=True, stat="density", ax=axes[1, 1])
axes[1, 1].set_title("width")

plt.tight_layout()
plt.show()

Эти распределения строго не нормальные, но ближе к нему, чем исходные, можно использовать при обучении логарифмированные таргеты

Также заметим экстремальные значения, посмотрим на них

In [ ]:
for c in target_cols:
    print(c)
    
    print("Cамые маленькие значения")
    display(df.nsmallest(10, c)[['title'] + target_cols])
    
    print("Cамые большие значения")
    display(df.nlargest(10, c)[['title'] + target_cols])

Видим плохую разметку, скорее всего будем получать плохие результаты в моделях, где выбросы сильно мешают

Проанализируем продавцов и покупателй, посчитаем сколько уникальных продавцов есть в датасете

In [ ]:
df['seller_id'].nunique() / df.shape[0], df['buyer_id'].nunique() / df.shape[0]

Большинство продавцов и покупателей уникальные, кажется эти признаки не будут нести большего смысла, уберем их

Поработаем с датами, посмотри с какого до какого временного промежутка заказывали товары

In [ ]:
dates = pd.to_datetime(df['order_date'])
dates.min(), dates.max()

Товары заказывались в течение всего 2024 года, посмотрим на распределение по сезонам, возможно будет полезная фича

In [ ]:
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Зима'
    elif month in [3, 4, 5]:
        return 'Весна'
    elif month in [6, 7, 8]:
        return 'Лето'
    else:
        return 'Осень'

seasons = dates.apply(get_season)

seasons.value_counts().reindex(['Зима', 'Весна', 'Лето', 'Осень']).plot(kind='bar')

plt.title('Количество заказов по временам года')
plt.ylabel('Количество')
plt.xticks(rotation=0)
plt.show()

Посмотрим по месяцам

In [ ]:
def get_month(date):
    month = date.month
    months = {
        1: 'Январь',
        2: 'Февраль',
        3: 'Март',
        4: 'Апрель',
        5: 'Май',
        6: 'Июнь',
        7: 'Июль',
        8: 'Август',
        9: 'Сентябрь',
        10: 'Октябрь',
        11: 'Ноябрь',
        12: 'Декабрь'
    }
    return months[month]

months_series = dates.apply(get_month)

months_order = [
    'Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь',
    'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'
]

months_series.value_counts().reindex(months_order).plot(kind='bar')
plt.title('Количество заказов по месяцам')
plt.ylabel('Количество')
plt.xticks(rotation=45)
plt.show()


Можно использовать как фичу, использовать будем только один, чтобы избежать мультиколлинерности

Посмотрим на цены

In [ ]:
df['item_price'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sns.boxplot(x="item_price", data=df, ax=axes[0])

sns.histplot(df["item_price"], kde=True, stat="density", ax=axes[1])

plt.tight_layout()
plt.show()


Видим очень странные товары по 1 рублю, посмотрим на эти товары, также отметим сильную скошенность данных, по аналогии с таргетом будем логарифмировать

In [ ]:
df[df['item_price'] == 1].sample(3, random_state=RANDOM_STATE)

Можно заметить, что это товары, продающиеся по акции #яПомогаю, эти товары выставляют за <10 рублей, посмотрим сколько их

In [ ]:
len(df[df['item_price'] < 10]) / len(df) * 100

Это всего лишь 0.03% процента от всех данных, заменим цену этих продуктов средним по микрокатегории и прологарифмируем

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sns.boxplot(x=np.log(df['item_price']), ax=axes[0])

sns.histplot(np.log(df["item_price"]), kde=True, stat="density", ax=axes[1])

plt.tight_layout()
plt.show()


Получили более нормальное распределние

Проанализируем категории

In [ ]:
categories = ['category_name', 'subcategory_name', 'microcat_name']

for cat in categories:
    print(cat, df[cat].nunique())

Посмотрим на распределения категорий и субкатегорий, также можно отметить что их недостаточно много, можно будет кодировать с помощью OHE

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

df['category_name'].value_counts().plot(kind='bar', ax=axes[0]);
df['subcategory_name'].value_counts().plot(kind='bar', ax=axes[1]);

Можно увидеть, что есть очень редкие субкатегории 

In [ ]:
df['subcategory_name'].value_counts()

Можно попробовать объеденить какие-то категории, например *Настольные компьютеры и ноутбуки*

Теперь посмотрим на микрокатегории

In [ ]:
df['microcat_name'].value_counts(normalize=True)

Самая частая категория встречается всего лишь в 3% категорий, если использовать OHE, то будет очень шумно, можно использовать Target Encoding

Теперь посмотрим на заголовки и описания

In [ ]:
df[['title', 'description']].sample(10, random_state=RANDOM_STATE)

Заголовки ингнформативны с точки зрения определения товара, в описаниях бывает информация о размерах, можно на этапе препроцессинга текста как-то выделять эти габариты и веса 

Теперь посмотрим на картинки

In [ ]:
target_cols = ['real_height', 'real_width', 'real_length', 'real_weight']
sample_df = df.sample(9, random_state=RANDOM_STATE)

fig, axes = plt.subplots(3, 3, figsize=(12, 12))
axes = axes.flatten()

for idx, row in enumerate(sample_df.itertuples()):
    img_path = os.path.join("data/train", row.image_name)
    
    img = Image.open(img_path)
    axes[idx].imshow(img)
    
    axes[idx].axis('off')
    
    target_text = ', '.join([f"{col}: {getattr(row, col)}" for col in target_cols])
    wrapped_text = "\n".join(textwrap.wrap(target_text, width=25))
    axes[idx].set_title(wrapped_text, fontsize=10)

plt.tight_layout(pad=3.0)
plt.show()

Картинки иногда довольно шумные, но можно заметить, что стоит аккуратно проводить аугментации с пропорциями и кропами, чтобы сеть уловила габариты

# Preprocessing

- Применим log к цене
- Сгенерим фичу сезона из даты
- Обработаем пропуски в состоянии товара и объединим повторяющиеся
- Сделаем One Hot и Target Encoding
- Предобработаем **title + description**, построим TF-IDF
- Получим эмбеддинги картинок через предпоследний слой ResNet50

Начнем с текстов, напишем функцию для предобработки: стемминг, приведем к нижнему регистру, поработаем с указанием габаритов в текстах

In [ ]:
stemmer = SnowballStemmer("russian")
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))

def preprocess(text):
    text = text.lower()
    text = re.sub(r'(\d+)\s*(см|сантиметр|сантиметров|cm)', r'\1см', text)
    text = re.sub(r'(\d+)\s*(мм|миллиметр|миллиметров|mm)', r'\1мм', text)
    text = re.sub(r'(\d+)\s*(кг|килограмм|килограммов|kg)', r'\1кг', text)
    text = re.sub(r'(\d+)\s*(гр|грамм|граммов|g)', r'\1гр', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'[^a-zа-яё0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    words = text.split()
    processed_words = [
        word if any(c.isdigit() for c in word) 
        else stemmer.stem(word)
        for word in words
        if word not in russian_stopwords
    ]
    
    return " ".join(processed_words)

In [ ]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        combined_text = X.iloc[:, 0].astype(str) + " " + X.iloc[:, 1].astype(str)
        
        res = combined_text.apply(preprocess)
        
        return res.values.astype(str)

Напишем экстрактор сезона и месяца из дат

In [ ]:
class SeasonExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): 
        return self
    
    def transform(self, X):
        dt = pd.to_datetime(X.iloc[:, 0])
        return dt.map(get_season).values.reshape(-1, 1)

In [ ]:
class MonthExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        dt = pd.to_datetime(X.iloc[:, 0])
        return dt.map(get_month).values.reshape(-1, 1)

Напишем класс для обработки колонки **item_condition**

In [ ]:
class ConditionCleaner(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): 
        return self
    
    def transform(self, X):
        return (
            X.fillna("Неизвестно")
            .replace({'Новый': 'Новое', 'Новое с биркой': 'Новое'})
            .values
            .reshape(-1, 1)
        )

Напишем класс для мульти таргет энкодинга

In [ ]:
class MultiTargetEncoding(BaseEstimator, TransformerMixin):
    def __init__(self, m=5.0):
        self.m = m
        self.encoders = []

    def fit(self, X, y):
        y = np.array(y)
        for i in range(y.shape[1]):
            enc = MEstimateEncoder(m=self.m)
            enc.fit(X, y[:, i])
            self.encoders.append(enc)
        return self

    def transform(self, X):
        encoded_cols = []
        for enc in self.encoders:
            encoded_cols.append(enc.transform(X))
        return np.hstack(encoded_cols)

Напишем класс для корректной обработки низких цен

In [ ]:
class PriceMicrocatImputer(BaseEstimator, TransformerMixin):
    def __init__(self, price_col='item_price', microcat_col='microcat_name', min_price=10):
        self.price_col = price_col
        self.microcat_col = microcat_col
        self.min_price = min_price

    def fit(self, X, y=None):
        X = X.copy()
        
        self.microcat_means_ = (
            X[X[self.price_col] >= self.min_price]
            .groupby(self.microcat_col)[self.price_col]
            .mean()
        )
        
        self.global_mean_ = (
            X[X[self.price_col] >= self.min_price][self.price_col].mean()
        )
        
        return self

    def transform(self, X):
        X = X.copy()
        
        mask = X[self.price_col] < self.min_price
        
        X.loc[mask, self.price_col] = (
            X.loc[mask, self.microcat_col]
            .map(self.microcat_means_)
            .fillna(self.global_mean_)
        )
        
        return X


Поработаем с картинкамми, напишем класс Datset, получим эмбеддинги, функцию для их сохранения, чтобы лишний раз не тратить бесплатное gpu)

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, names, img_dir, transform, targets=None): 
        self.names = names
        self.img_dir = img_dir
        self.transform = transform
        self.targets = targets
        
    def __len__(self): 
        return len(self.names)
    
    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.img_dir, self.names[idx])).convert('RGB')
        img = self.transform(img)
        
        if self.targets is not None:
            return img, torch.tensor(self.targets[idx], dtype=torch.float32)
        
        return img

def get_embeddings(df, img_dir, filename, batch_size=128):
    weights = ResNet50_Weights.DEFAULT
    model = models.resnet50(weights=weights)
    model.fc = nn.Identity()
    model = model.to(DEVICE).eval()
    
    transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    
    dataset = ImageDataset(df['image_name'].values, img_dir, transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    all_embs = []
    with torch.no_grad():
        for batch in loader:
            embs = model(batch.to(DEVICE))
            all_embs.append(embs.cpu().numpy())
    
    res = np.vstack(all_embs)
    np.save(f'{filename}.npy', res)
    return res

Подготовим данные, разобъем на тренировочный и валидационный датасет 

In [ ]:
target_cols = ['real_weight', 'real_height', 'real_length', 'real_width']

X_train, X_val, y_train, y_val = train_test_split(
    df.drop(columns=target_cols), df[target_cols].values, test_size=0.1, random_state=RANDOM_STATE
)

Получим эмбеддинги картинок и снизим размерность, чтобы избежать проблем с памятью

In [ ]:
# train_embeddings = get_embeddings(X_train, '/kaggle/input/aaa-exam/train', 'train_embeddings')
# val_embeddings = get_embeddings(X_val, '/kaggle/input/aaa-exam/train', 'val_embeddings')
train_resnet_embeddings = np.load('embeddings/train_resnet_embeddings.npy')
val_resnet_embeddings = np.load('embeddings/val_resnet_embeddings.npy')

Cнизим размерность с помощью PCA так, чтобы сохранилось 90% дисперсии

In [ ]:
pca = PCA(n_components=0.9, random_state=RANDOM_STATE)
train_resnet_emb_reduced = pca.fit_transform(train_resnet_embeddings)
val_resnet_emb_reduced = pca.transform(val_resnet_embeddings)

In [ ]:
print("Исходная размерность:", train_resnet_embeddings.shape[1])
print("Новая размерность:", train_resnet_emb_reduced.shape[1])

In [ ]:
pipeline = ColumnTransformer([

    ('month', Pipeline([
        ('ext', MonthExtractor()),
        ('ohe', OneHotEncoder(handle_unknown='ignore'))
    ]), ['order_date']),

    ('condition', Pipeline([
        ('clean', ConditionCleaner()),
        ('ohe', OneHotEncoder(handle_unknown='ignore'))
    ]), 'item_condition'),

    ('price', Pipeline([
        ('impute', PriceMicrocatImputer(
            price_col='item_price',
            microcat_col='microcat_name',
            min_price=10
        )),
        ('select', FunctionTransformer(lambda X: X[['item_price']], validate=False)),
        ('log', FunctionTransformer(np.log1p))
    ]), ['item_price', 'microcat_name']),

    ('cats', OneHotEncoder(handle_unknown='ignore'),
     ['category_name', 'subcategory_name']),

    ('microcat_te', MultiTargetEncoding(), 'microcat_name'),

    ('text', Pipeline([
        ('prep', TextPreprocessor()),
        ('tfidf', TfidfVectorizer(
            ngram_range=(1, 2),
            min_df=5,
            max_df=0.9,
            max_features=5_000
        ))
    ]), ['title', 'description'])

], sparse_threshold=0.3, verbose=True)


# Подбор модели
## Baseline
Получим матрицу для обучения, в качестве бейзлайна используем Ridge

In [ ]:
X_train_processed = pipeline.fit_transform(X_train, y_train)
X_val_processed = pipeline.transform(X_val)

In [ ]:
X_train_final = sp.hstack([
    X_train_processed, 
    sp.csr_matrix(train_resnet_emb_reduced)
]).tocsr()

X_val_final = sp.hstack([
    X_val_processed, 
    sp.csr_matrix(val_resnet_emb_reduced)
]).tocsr()

Проведем нормализацию

In [ ]:
scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train_final)
X_val_scaled = scaler.transform(X_val_final)

Подбор гиперпараметров

In [ ]:
ridge_base = Ridge(solver='sparse_cg', max_iter=500)

param_grid = {'alpha': [0.1, 1.0, 5.0, 20.0, 50.0]}

grid_search = GridSearchCV(
    ridge_base,
    param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train_scaled, np.log1p(y_train))
best_ridge = grid_search.best_estimator_

In [ ]:
grid_search.best_params_

In [ ]:
y_pred = np.expm1(best_ridge.predict(X_val_scaled))

for i, col in enumerate(target_cols):
    logmae = log_mae(y_val[:, i], y_pred[:, i])
    mae = mean_absolute_error(y_val[:, i], y_pred[:, i])
    rmse = np.sqrt(mean_squared_error(y_val[:, i], y_pred[:, i]))
    r2 = r2_score(y_val[:, i], y_pred[:, i])
    
    print(f"\nTarget: {col}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R2:   {r2:.4f}")
    print(f"  LogMAE:   {logmae:.4f}")
    
print(f"Macro LogMAE: {macro_log_mae(y_val, y_pred)}")

Видим последствия тех самых выбросов, в целом по логарифмам и MAE результат приемлимый

## SBERT
Изменим представление текста: получим эмбеддинги текстов с помошью ruBERT отдельно для текста и описания

In [ ]:
def get_bert_embeddings_by_column(df, filename_prefix, batch_size=64):
    model = SentenceTransformer('rubert-tiny2')

    title_texts = df['title'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip().tolist()
    description_texts = df['description'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip().tolist()

    title_emb = model.encode(title_texts, batch_size=batch_size, device=DEVICE, show_progress_bar=True, convert_to_numpy=True)
    description_emb = model.encode(description_texts, batch_size=batch_size, device=DEVICE, show_progress_bar=True, convert_to_numpy=True)

    np.save(f'{filename_prefix}_title.npy', title_emb)
    np.save(f'{filename_prefix}_description.npy', description_emb)

    return title_emb, description_emb

In [ ]:
# title_emb_train, desc_emb_train = get_bert_embeddings_by_column(X_train, 'sbert_train', batch_size=128)
# title_emb_val, desc_emb_val = get_bert_embeddings_by_column(X_val, 'sbert_val', batch_size=128)

title_emb_train = np.load('embeddings/sbert_train_title.npy')
desc_emb_train  = np.load('embeddings/sbert_train_description.npy')
title_emb_val   = np.load('embeddings/sbert_val_title.npy')
desc_emb_val    = np.load('embeddings/sbert_val_description.npy')

In [ ]:
pca_title = PCA(n_components=0.9, random_state=RANDOM_STATE)
title_emb_train_reduced = pca_title.fit_transform(title_emb_train)
title_emb_val_reduced = pca_title.transform(title_emb_val)

pca_desc = PCA(n_components=0.9, random_state=RANDOM_STATE)
desc_emb_train_reduced = pca_desc.fit_transform(desc_emb_train)
desc_emb_val_reduced = pca_desc.transform(desc_emb_val)

In [ ]:
pipeline_without_tfidf = ColumnTransformer([
    ('month', Pipeline([
        ('ext', MonthExtractor()),
        ('ohe', OneHotEncoder(handle_unknown='ignore'))
    ]), ['order_date']),

    ('condition', Pipeline([
        ('clean', ConditionCleaner()),
        ('ohe', OneHotEncoder(handle_unknown='ignore'))
    ]), 'item_condition'),

    ('price', Pipeline([
        ('impute', PriceMicrocatImputer(
            price_col='item_price',
            microcat_col='microcat_name',
            min_price=10
        )),
        ('select', FunctionTransformer(lambda X: X[['item_price']], validate=False)),
        ('log', FunctionTransformer(np.log1p))
    ]), ['item_price', 'microcat_name']),

    ('cats', OneHotEncoder(handle_unknown='ignore'),
     ['category_name', 'subcategory_name']),

    ('microcat_te', MultiTargetEncoding(), 'microcat_name'),
], sparse_threshold=0.0, verbose=True)


In [ ]:
X_train_raw = pipeline_without_tfidf.fit_transform(X_train, y_train)
X_val_raw = pipeline_without_tfidf.transform(X_val)

In [ ]:
X_train_stacked = np.hstack([
    X_train_raw,
    title_emb_train_reduced, 
    desc_emb_train_reduced,
    train_resnet_emb_reduced,
])

X_val_stacked = np.hstack([
    X_val_raw,
    title_emb_val_reduced, 
    desc_emb_val_reduced,
    val_resnet_emb_reduced,
])

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_stacked)
X_val_scaled = scaler.transform(X_val_stacked)

In [ ]:
ridge_base = Ridge()

param_grid = {'alpha': [0.1, 1.0, 5.0, 20.0, 50.0, 75.0]}

grid_search = GridSearchCV(
    ridge_base,
    param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train_scaled, np.log1p(y_train))
best_ridge = grid_search.best_estimator_

In [ ]:
grid_search.best_params_

In [ ]:
y_pred = np.expm1(best_ridge.predict(X_val_scaled))

for i, col in enumerate(target_cols):
    logmae = log_mae(y_val[:, i], y_pred[:, i])
    mae = mean_absolute_error(y_val[:, i], y_pred[:, i])
    rmse = np.sqrt(mean_squared_error(y_val[:, i], y_pred[:, i]))
    r2 = r2_score(y_val[:, i], y_pred[:, i])
    
    print(f"\nTarget: {col}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R2:   {r2:.4f}")
    print(f"  LogMAE:   {logmae:.4f}")
    
print(f"Macro LogMAE: {macro_log_mae(y_val, y_pred)}")

Видим,что результат даже стал немного хуже, хотя качество эмбеддингов должно быть лучше чем TF-IDF

## Catboost
Попробуем CatBoost

In [ ]:
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

model = CatBoostRegressor(
    iterations=3000,
    learning_rate=0.05,
    depth=8,
    loss_function='MultiRMSE',
    eval_metric='MultiRMSE',
    early_stopping_rounds=200,
    l2_leaf_reg=4,
    verbose=200,
    task_type="GPU",
    random_seed=RANDOM_STATE
)

model.fit(
    X_train_stacked, y_train_log,
    eval_set=(X_val_stacked, y_val_log)
)

y_pred_log = model.predict(X_val_stacked)
y_pred = np.expm1(y_pred_log)

In [ ]:
for i, col in enumerate(target_cols):
    mae = mean_absolute_error(y_val[:, i], y_pred[:, i])
    rmse = np.sqrt(mean_squared_error(y_val[:, i], y_pred[:, i]))
    r2 = r2_score(y_val[:, i], y_pred[:, i])
    l_mae = log_mae(y_val[:, i], y_pred[:, i])
    
    print(f"\nTarget: {col}")
    print(f"  MAE: {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R2: {r2:.4f}")
    print(f"  LogMAE: {l_mae:.4f}")

print(f"\nMacro LogMAE: {macro_log_mae(y_val, y_pred)}")

Видим, то что logMAE снизился

Попробуем воспользоваться встроенной возможностью катбуста кодировать категориальные переменные

In [ ]:
X_train_final.to_pickle("X_train_final.pkl")
X_val_final.to_pickle("X_val_final.pkl")

In [ ]:
def prepare_for_catboost(df, title_emb, desc_emb, img_emb, min_price=10, price_clip=(10, 100_000)):
    res = df[['item_price', 'category_name', 'subcategory_name', 'microcat_name']].copy()

    microcat_means = (
        res[res['item_price'] >= min_price]
        .groupby('microcat_name')['item_price']
        .mean()
        .to_dict()
    )
    global_mean = res[res['item_price'] >= min_price]['item_price'].mean()
    
    def impute_price(row):
        if row['item_price'] < min_price:
            return microcat_means.get(row['microcat_name'], global_mean)
        return row['item_price']
    
    res['item_price'] = res.apply(impute_price, axis=1)
    res['item_price'] = np.log1p(res['item_price'])
    res['item_price'] = res['item_price'].clip(np.log1p(price_clip[0]), np.log1p(price_clip[1]))

    res['item_condition'] = df['item_condition'].fillna("Неизвестно").replace({
        'Новый': 'Новое',
        'Новое с биркой': 'Новое',
    }).astype(str)

    dates = pd.to_datetime(df['order_date'])
    res['month'] = dates.dt.month.astype(str)

    for col in ['category_name', 'subcategory_name', 'microcat_name', 'item_condition', 'month']:
        res[col] = res[col].astype(str)

    res = res.reset_index(drop=True)

    title_df = pd.DataFrame(title_emb).add_prefix('title_e_')
    desc_df = pd.DataFrame(desc_emb).add_prefix('desc_e_')
    img_df = pd.DataFrame(img_emb).add_prefix('img_e_')

    final_df = pd.concat([res, title_df, desc_df, img_df], axis=1)

    return final_df

X_train_final = prepare_for_catboost(X_train, title_emb_train_reduced, desc_emb_train_reduced, train_resnet_emb_reduced)
X_val_final = prepare_for_catboost(X_val, title_emb_val_reduced, desc_emb_val_reduced, val_resnet_emb_reduced)

In [ ]:
cat_cols = ['category_name', 'subcategory_name', 'microcat_name', 'item_condition', 'month']


model = CatBoostRegressor(
    iterations=3000,
    learning_rate=0.05,
    depth=8,
    loss_function='MultiRMSE',
    eval_metric='MultiRMSE',
    early_stopping_rounds=200,
    l2_leaf_reg=4,
    verbose=200,
    task_type="GPU",
    random_seed=RANDOM_STATE
)
    
model.fit(
    X_train_final, y_train_log,
    cat_features=cat_cols,
    eval_set=(X_val_final, y_val_log)
)
    
y_pred_log = model.predict(X_val_final)
y_pred = np.expm1(y_pred_log)

In [ ]:
for i, col in enumerate(target_cols):
    mae = mean_absolute_error(y_val[:, i], y_pred[:, i])
    rmse = np.sqrt(mean_squared_error(y_val[:, i], y_pred[:, i]))
    r2 = r2_score(y_val[:, i], y_pred[:, i])
    l_mae = log_mae(y_val[:, i], y_pred[:, i])
    
    print(f"\nTarget: {col}")
    print(f"  MAE: {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R2: {r2:.4f}")
    print(f"  LogMAE: {l_mae:.4f}")

print(f"\nMacro LogMAE: {macro_log_mae(y_val, y_pred)}")

Видим очень незначительную разницу, ручное кодироваание привело даже к лучшим результатам

# Тест

In [ ]:
model = CatBoostRegressor()
model.load_model("catboost_model.cbm");

In [ ]:
X_test = pipeline_without_tfidf.transform(test_df)

In [ ]:
title_emb_test_reduced = pca_title.transform(np.load('embeddings/sbert_test_title.npy'))
desc_emb_test_reduced = pca_desc.transform(np.load('embeddings/sbert_test_description.npy'))
test_resnet_emb_reduced = pca.transform(np.load('embeddings/test_resnet_embeddings.npy'))

In [ ]:
X_test_stacked = np.hstack([
    X_test,
    title_emb_test_reduced, 
    desc_emb_test_reduced,
    test_resnet_emb_reduced,
])

In [ ]:
X_test_final = prepare_for_catboost(test_df, title_emb_test_reduced, desc_emb_test_reduced, test_resnet_emb_reduced)

In [ ]:
y_pred = np.expm1(model.predict(X_test_stacked))

In [ ]:
predictions_df = pd.DataFrame(y_pred, columns=['weight', 'height', 'length', 'width'])
predictions_df['item_id'] = test_df['item_id'].values

predictions_df = predictions_df[['item_id', 'weight', 'height', 'length', 'width']]

predictions_df.to_csv('predictions.csv', index=False)

Score на Stepik: **0.327985**